In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
trials_data= pd.read_csv("trials_training_data.csv")

In [3]:
trials_data

,Name,State,Notes,User,Tags,Created,Runtime,Sweep,base_lr,epochs,...,loss/val,lr,precip_val_loss,temp_val_loss,tmax_val_loss,tmin_val_loss,total_val_loss,trial,weights,initial_weights
0,UNCONSTRAINED_trial_third_iter_36,finished,-,NaN,NaN,2025-07-29T11:07:03.000Z,6887,NaN,NaN,NaN,...,0.000080,0.0008,0.000006,0.000059,0.000087,0.000088,0.000065,36,"[0.1655939840186998,0.32269589944483285,0.3281...","[0.2213762562577586,0.4313997911950722,0.43867..."
1,UNCONSTRAINED_trial_third_iter_35,finished,-,NaN,NaN,2025-07-29T09:12:43.000Z,6860,NaN,NaN,NaN,...,0.000082,0.0010,0.000007,0.000070,0.000129,0.000094,0.000072,35,"[0.18629607350063965,0.3713466092501947,0.3578...","[0.4562693695632175,0.9094882146909182,0.87634..."
2,UNCONSTRAINED_trial_third_iter_34,finished,-,NaN,NaN,2025-07-29T07:17:55.000Z,6887,NaN,NaN,NaN,...,0.000067,0.0008,0.000004,0.000091,0.000143,0.000195,0.000067,34,"[0.4501022934021185,0.33115694433582243,0.0752...","[0.9022785787960418,0.663839801471137,0.150924..."
3,UNCONSTRAINED_trial_third_iter_33,finished,-,NaN,NaN,2025-07-29T05:24:02.000Z,6832,NaN,NaN,NaN,...,0.000074,0.0010,0.000006,0.000082,0.000094,0.000083,0.000074,33,"[0.1473106772300055,0.12779418007377627,0.5112...","[0.25151409835012206,0.21819218117817468,0.872..."
4,UNCONSTRAINED_trial_third_iter_32,finished,-,NaN,NaN,2025-07-29T03:30:01.000Z,6841,NaN,NaN,NaN,...,0.000080,0.0010,0.000007,0.000069,0.000098,0.000097,0.000080,32,"[0.06523571476095563,0.39579171496233495,0.312...","[0.14414210471161432,0.8745248064061308,0.6901..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,trial_5,finished,-,NaN,NaN,2025-07-20T17:38:06.000Z,7075,NaN,NaN,NaN,...,0.000051,0.0008,0.000005,0.000058,0.000072,0.000087,0.000051,8,"[0.3118304899680882,0.2532597408016314,0.20394...",NaN
94,trial_3,finished,-,NaN,NaN,2025-07-20T15:36:19.000Z,7301,NaN,NaN,NaN,...,0.000062,0.0010,0.000005,0.000066,0.000114,0.000103,0.000059,4,"[0.36503006197686494,0.2575344108920214,0.2530...",NaN
95,trial_0,finished,-,NaN,NaN,2025-07-20T10:31:40.000Z,17670,NaN,NaN,NaN,...,0.000054,0.0008,0.000004,0.000056,0.000084,0.000098,0.000054,0,"[0.2886978837916178,0.3378427925582116,0.16448...",NaN
96,trial_4,finished,The runtime is less because the run was initia...,NaN,NaN,2025-07-19T16:53:35.000Z,6,NaN,0.0001,20.0,...,0.000058,0.0008,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN


In [4]:
trials_data.columns

Index(['Name', 'State', 'Notes', 'User', 'Tags', 'Created', 'Runtime', 'Sweep',
       'base_lr', 'epochs', 'loss', 'loss_weights', 'max_lr', 'mode',
       'optimizer', 'optuna_trial', 'scheduler', 'RhiresD/train',
       'RhiresD/val', 'TabsD/train', 'TabsD/val', 'TmaxD/train', 'TmaxD/val',
       'TminD/train', 'TminD/val', 'best_val_loss',
       'best_val_loss_per_channel', 'epoch', 'epoch_time', 'loss/train',
       'loss/val', 'lr', 'precip_val_loss', 'temp_val_loss', 'tmax_val_loss',
       'tmin_val_loss', 'total_val_loss', 'trial', 'weights',
       'initial_weights'],
      dtype='object')

In [ ]:
y = trials_data['RhiresD/val'].values
x = trials_data['loss/val'].values

def pareto_front_2d(x, y):
    # Sort by x (first objective)
    idx_sorted = np.argsort(x)
    pareto_idx = []
    min_y = np.inf
    for idx in idx_sorted:
        if y[idx] < min_y:
            pareto_idx.append(idx)
            min_y = y[idx]
    pareto_points = np.array(list(zip(x[pareto_idx], y[pareto_idx])))
    return pareto_points, np.array(pareto_idx)

pareto_points, pareto_idx = pareto_front_2d(x, y)

plt.figure(figsize=(12, 8))
plt.scatter(x, y, label='Trials (98)', color='blue', alpha=0.5, s=50)
# Annotate Pareto "elbow" (closest to origin)
pareto_distances = np.sqrt(pareto_points[:,0]**2 + pareto_points[:,1]**2)
elbow_idx = np.argmin(pareto_distances)
elbow_x, elbow_y = pareto_points[elbow_idx]
elbow_trial_idx = pareto_idx[elbow_idx]
elbow_trial_name = trials_data.iloc[elbow_trial_idx]['Name']
plt.scatter(elbow_x, elbow_y, marker='*', s=200, color='red', label='Pareto elbow')
plt.annotate(f'{elbow_trial_name}', (elbow_x, elbow_y),
             textcoords="offset points", xytext=(10,-20), ha='left', color='red', fontsize=14, fontweight='bold')

plt.xlabel('Total validation loss')
plt.ylabel('Precipitation Val Loss')
plt.title('Total validation loss vs Precipitation Val Loss for "constrained" trials')
plt.legend()
plt.grid(False)
plt.tight_layout()
plt.savefig('pareto_front_plot.png', dpi=500, bbox_inches='tight')